In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/kneefinal.zip"

Streaming output truncated to the last 5000 lines.
  inflating: train/0/9684605R.png    
  inflating: train/0/9685238L.png    
  inflating: train/0/9685238R.png    
  inflating: train/0/9686617L.png    
  inflating: train/0/9686617R.png    
  inflating: train/0/9686777R.png    
  inflating: train/0/9686834L.png    
  inflating: train/0/9686834R.png    
  inflating: train/0/9686908L.png    
  inflating: train/0/9686908R.png    
  inflating: train/0/9687273L.png    
  inflating: train/0/9689906R.png    
  inflating: train/0/9690910L.png    
  inflating: train/0/9690910R.png    
  inflating: train/0/9691359L.png    
  inflating: train/0/9691359R.png    
  inflating: train/0/9692163L.png    
  inflating: train/0/9692604L.png    
  inflating: train/0/9692604R.png    
  inflating: train/0/9692923R.png    
  inflating: train/0/9693364R.png    
  inflating: train/0/9693599L.png    
  inflating: train/0/9693599R.png    
  inflating: train/0/9694101L.png    
  inflating: train/0/9694101R.png    

In [3]:
import tensorflow as tf #keras-deep learning library library built on top of tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation #dense layer is fully connected layer.
#Dropout is to prevent your model from overfitting.
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.densenet import preprocess_input

In [4]:
image_width,image_height=224,224 #height and width of acceptable images to the model
training_data_dir=r"/content/train"
validation_data_dir= r"/content/val"
testing_data_dir=r"/content/test"
epochs=70 #70 times the model will learn from the data.
batch_size=48  #takes 20 samples from the dataset

In [5]:
import tensorflow.keras.backend as k
if k.image_data_format()=="channels_first":
  input_shape=(3,image_width,image_height)
else:
  input_shape=(image_width,image_height,3)

In [6]:
trained_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 rescale=1./255, #rescaling the pixels in range 0-255 to 0-1.
                                #  shear_range=0.2, #applies 20% shear strain over image .
                                 zoom_range=0.3, #adjust the zoom in the range of 1-0.3 to 1+0.3.
                                #  horizontal_flip=True,vertical_flip=True, #flips the image vertically and horizontally.
                                #  rotation_range=360, #rotates the image in range of 0 to 360.
                                 fill_mode='nearest', #after rotating, if the image has whitespaces created then they are filled with the nearest pixels.
                                 width_shift_range=0.2,height_shift_range=0.2, #shifts the image by 20% in horizontal and vertical directions.
                                 brightness_range=[0.4,1.5] #this are the brightness shift values range
                                 )

validated_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1./255, brightness_range=[0.4,1.5])
trained_generator=trained_datagen.flow_from_directory(training_data_dir,target_size=(image_width,image_height),batch_size=batch_size,class_mode="categorical",
                                                  classes=["0","1","2","3","4"])

validated_generator=validated_datagen.flow_from_directory(validation_data_dir,target_size=(image_width,image_height),batch_size=batch_size,class_mode="categorical",
                                                      classes=["0","1","2","3","4"])

Found 5778 images belonging to 5 classes.
Found 826 images belonging to 5 classes.


In [7]:
model_d=DenseNet169(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    # classes=3,
    # classifier_activation="softmax",
)
model_d.summary()

51877672/51877672 [==============================] - 3s 0us/step
Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormali

In [ ]:
model_x.trainable=True
set_trainable=False

for layer in model_x.layers:
  if layer.name=='conv2d_12':
    set_trainable=True
  if set_trainable:
    layer.trainable=True
  else:
    layer.trainable=False

for layer in model_x.layers:
  print(layer.name,layer.trainable)

In [8]:
from keras.models import Model

In [9]:
#making classification layer to fit our model
from matplotlib.cbook import flatten
from tensorflow.keras import layers, models

x=model_d.layers[-3].output

x=tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding="same")(x)
x=tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Activation("relu")(x)

x=tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same")(x)
x=tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Activation("relu")(x)

x=tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same")(x)
x=tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Activation("relu")(x)

x=tf.keras.layers.Conv2D(filters=5, kernel_size=3, padding="same")(x)
x=tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Activation("relu")(x)

GAP = tf.keras.layers.GlobalAveragePooling2D()(x)
pred = tf.keras.activations.softmax(GAP)

densenet_model = Model(inputs=model_d.input,outputs=pred)
# flatten_layer=layers.Flatten()
# dense_layer1=layers.Dense(50,activation='relu')

# dense_layer2=layers.Dense(20,activation='relu')

# predictionlayer=layers.Dense(3,activation='softmax')

# model_I17=models.Sequential([model_I1,
#                          flatten_layer,
#                          dense_layer1,
#                         Dropout(0.2),
#                          dense_layer2,
#                          Dropout(0.2),
#                          predictionlayer]) #dropout after each dense layer deactivates the 20% nodes of the above layer randomly in each epoch.
# model_I17.summary()

In [10]:
from keras.optimizers import Adam


densenet_model.compile(optimizer = "Adam",
                 metrics=["acc"],
                 loss= tf.keras.losses.categorical_crossentropy)

In [11]:
from keras.callbacks import ReduceLROnPlateau
reduced_lr= ReduceLROnPlateau(monitor='val_loss',factor=0.2, patience=5, min_lr=0.0001)
callback_list=[reduced_lr]

In [12]:
hist1=densenet_model.fit(trained_generator,epochs=epochs, validation_data=validated_generator,shuffle=True, callbacks=callback_list)

Epoch 1/70
121/121 [==============================] - 250s 1s/step - loss: 1.4549 - acc: 0.4168 - val_loss: 1.6691 - val_acc: 0.2567 - lr: 0.0010
Epoch 2/70
121/121 [==============================] - 121s 998ms/step - loss: 1.3404 - acc: 0.5021 - val_loss: 1.5317 - val_acc: 0.3971 - lr: 0.0010
Epoch 3/70
121/121 [==============================] - 120s 986ms/step - loss: 1.2752 - acc: 0.5268 - val_loss: 1.6230 - val_acc: 0.2567 - lr: 0.0010
Epoch 4/70
121/121 [==============================] - 120s 985ms/step - loss: 1.2391 - acc: 0.5429 - val_loss: 1.6638 - val_acc: 0.1295 - lr: 0.0010
Epoch 5/70
121/121 [==============================] - 120s 992ms/step - loss: 1.1910 - acc: 0.5569 - val_loss: 1.5675 - val_acc: 0.3002 - lr: 0.0010
Epoch 6/70
121/121 [==============================] - 120s 990ms/step - loss: 1.1585 - acc: 0.5689 - val_loss: 1.4148 - val_acc: 0.4213 - lr: 0.0010
Epoch 7/70
121/121 [==============================] - 120s 985ms/step - loss: 1.1210 - acc: 0.5768 - val_loss

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#list all data in training
print(his2.history.keys())
#summarize training for accuracy
plt.plot(his2.history['acc'])
plt.plot(his2.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc="upper left")
plt.show()
#summarize training for loss
plt.plot(his2.history['loss'])
plt.plot(his2.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()